# Spam detector example - Multinomial Naive Bayes

In [1]:
import sys

!{sys.executable} -m pip install skl2onnx
!{sys.executable} -m pip install onnx
!{sys.executable} -m pip install onnxruntime

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
from skl2onnx import to_onnx
from skl2onnx.common.data_types import StringTensorType
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

import onnxruntime as rt
import pandas as pd

df = pd.read_csv("spam.csv", encoding="L1")

X = df["v2"].values
y = df["v1"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=345)

# create a pipeline object
pipe = make_pipeline(
    CountVectorizer(binary=True),
    MultinomialNB()
)

# fit the whole pipeline
pipe.fit(X_train, y_train)

# we can now use it like any other estimator
print(accuracy_score(pipe.predict(X_test), y_test))

onxx_pipeline = to_onnx(pipe, initial_types=[('message', StringTensorType([None, 1]))])

with open("spam_nb.onnx", "wb") as f:
    f.write(onxx_pipeline.SerializeToString())

0.9870782483847811


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/skl2onnx/operator_converters/text_vectoriser.py:179: UserWarning: Converter for TfidfVectorizer will use scikit-learn regular expression by default in version 1.6.
  UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/skl2onnx/common/_container.py:569: UserWarning: Unable to find operator 'Tokenizer' in domain 'com.microsoft' in ONNX, op_version is forced to 1.
  op_type, domain))


In [5]:
sess = rt.InferenceSession("spam_nb.onnx")
inputs = {'message': [["SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"]]}

pred_onx = sess.run(None, inputs)

In [6]:
print("prediction", pred_onx[0])
print("probabilites", pred_onx[1])

prediction ['spam']
probabilites [{'ham': 8.615117129305013e-23, 'spam': 1.0}]
